In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.7.3")
@file:DependsOn("org.apache.commons:commons-math3:3.6.1")
%use kandy
%use coroutines

import kotlin.math.*
import kotlin.system.measureTimeMillis

// Data class to hold benchmark results
data class BenchmarkResult(
    val method: String,
    val dataSize: Int,
    val timeMs: Double,
    val mse: Double,
    val compressionRatio: Double
)

class QuantizationBenchmark(
    private val sizes: List<Int> = listOf(1000, 10000, 100000, 1000000)
) {
    // Generate test data with mixed distributions
    private fun generateTestData(size: Int): FloatArray {
        val random = java.util.Random(42) // For reproducibility
        val normalDist = NormalDistribution(0.0, 1.0)
        
        return FloatArray(size) { i ->
            when (i) {
                // First third: sine wave
                in 0 until size/3 -> {
                    sin(i * 10.0f / (size/3)).toFloat()
                }
                // Second third: normal distribution
                in size/3 until 2*size/3 -> {
                    normalDist.sample().toFloat()
                }
                // Last third: uniform distribution
                else -> {
                    random.nextFloat() * 2 - 1 // Range [-1, 1]
                }
            }
        }
    }

    // Benchmark our custom quantizer
    private suspend fun benchmarkOurQuantizer(data: FloatArray): Pair<Double, Double> {
        var mse = 0.0
        var timeMs = 0.0
        
        withContext(Dispatchers.Default) {
            timeMs = measureTimeMillis {
                val quantized = Quantization.quantize8bit(data)
                val dequantized = Quantization.dequantize(quantized)
                
                // Calculate MSE
                mse = data.zip(dequantized).map { (a, b) -> 
                    (a - b) * (a - b) 
                }.average()
            }.toDouble()
        }
        
        return timeMs to mse
    }

    // Benchmark blockwise quantizer
    private suspend fun benchmarkBlockwiseQuantizer(data: FloatArray): Pair<Double, Double> {
        var mse = 0.0
        var timeMs = 0.0
        
        withContext(Dispatchers.Default) {
            timeMs = measureTimeMillis {
                val quantized = BlockwiseQuantization.quantizeBlockwise(
                    input = data,
                    blockSize = 1024
                )
                val dequantized = Quantization.dequantize(quantized)
                
                // Calculate MSE
                mse = data.zip(dequantized).map { (a, b) -> 
                    (a - b) * (a - b) 
                }.average()
            }.toDouble()
        }
        
        return timeMs to mse
    }

    // Run all benchmarks
    suspend fun runBenchmarks(): List<BenchmarkResult> = coroutineScope {
        val results = mutableListOf<BenchmarkResult>()
        
        for (size in sizes) {
            println("\\nBenchmarking with data size: $size")
            val data = generateTestData(size)
            
            // Our basic quantizer
            val (timeMs1, mse1) = benchmarkOurQuantizer(data)
            results.add(BenchmarkResult(
                "Basic Quantizer",
                size,
                timeMs1,
                mse1,
                4.0
            ))
            
            // Our blockwise quantizer
            val (timeMs2, mse2) = benchmarkBlockwiseQuantizer(data)
            results.add(BenchmarkResult(
                "Blockwise Quantizer",
                size,
                timeMs2,
                mse2,
                4.0
            ))
        }
        
        results
    }

    // Plot results using lets-plot
    fun plotResults(results: List<BenchmarkResult>) {
        // Convert results to format for plotting
        val data = mapOf(
            "Method" to results.map { it.method },
            "Data Size" to results.map { it.dataSize },
            "Time (ms)" to results.map { it.timeMs },
            "MSE" to results.map { it.mse }
        )

        // Time vs Data Size plot
        val timePlot = letsPlot(data) {
            x = "Data Size"
            y = "Time (ms)"
            color = "Method"
        } + 
        geomLine(size = 1.5) +
        geomPoint(size = 3.0) +
        scaleXLog10() +
        scaleYLog10() +
        themeLight() +
        labs(
            title = "Quantization Time vs Data Size",
            x = "Data Size (elements)",
            y = "Time (milliseconds)"
        )

        // MSE vs Data Size plot
        val msePlot = letsPlot(data) {
            x = "Data Size"
            y = "MSE"
            color = "Method"
        } + 
        geomLine(size = 1.5) +
        geomPoint(size = 3.0) +
        scaleXLog10() +
        scaleYLog10() +
        themeLight() +
        labs(
            title = "Mean Squared Error vs Data Size",
            x = "Data Size (elements)",
            y = "MSE"
        )

        // Print summary statistics
        println("\nSummary Statistics:")
        results.groupBy { it.method }.forEach { (method, methodResults) ->
            println("\n$method:")
            println("Average Time: %.2f ms".format(methodResults.map { it.timeMs }.average()))
            println("Average MSE: %.6f".format(methodResults.map { it.mse }.average()))
        }

        // Display plots
        timePlot.show()
        msePlot.show()
    }
}

// Run the benchmark
suspend fun main() {
    val benchmark = QuantizationBenchmark()
    val results = benchmark.runBenchmarks()
    benchmark.plotResults(results)
}

The problem is found in one of the loaded libraries: check library imports, dependencies and repositories
Error compiling code:
@file:Repository("https://packages.jetbrains.team/maven/p/kds/kotlin-ds-maven")
@file:DependsOn("org.jetbrains.kotlinx:kandy-lets-plot:0.6.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlin-statistics-jvm:0.2.1")


Errors:
Failed to resolve [Dependency(value=org.jetbrains.kotlinx:kandy-lets-plot:0.6.0, hasSources=MAYBE, isMultiplatform=NO), Dependency(value=org.jetbrains.kotlinx:kotlin-statistics-jvm:0.2.1, hasSources=MAYBE, isMultiplatform=NO)]:
File 'org.jetbrains.kotlinx:kandy-lets-plot:0.6.0' not found
File 'org.jetbrains.kotlinx:kotlin-statistics-jvm:0.2.1' not found
DependencyCollectionException: java.lang.InterruptedException
Unhandled exception during resolve

org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryException: The problem is found in one of the loaded libraries: check library imports, dependencies and repositories
	at org.jetbrains.kotlinx.j